In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
orders = pd.read_csv('../data/raw/orders.csv')
products = pd.read_csv('../data/raw/products.csv')
aisles = pd.read_csv('../data/raw/aisles.csv')
departments = pd.read_csv('../data/raw/departments.csv')
order_products_prior = pd.read_csv('../data/raw/order_products__prior.csv')
order_products_train = pd.read_csv('../data/raw/order_products__train.csv')

print("Data loaded!")
print(f"Orders: {orders.shape}")
print(f"Products: {products.shape}")
print(f"Order Products (prior): {order_products_prior.shape}")
print(f"Order Products (train): {order_products_train.shape}")

Data loaded!
Orders: (3421083, 7)
Products: (49688, 4)
Order Products (prior): (32434489, 4)
Order Products (train): (1384617, 4)


In [2]:
# Let's see what columns we have in each dataset
print("ORDERS columns:")
print(orders.head())
print("\n" + "="*80 + "\n")

print("PRODUCTS columns:")
print(products.head())
print("\n" + "="*80 + "\n")

print("AISLES columns:")
print(aisles.head())
print("\n" + "="*80 + "\n")

print("DEPARTMENTS columns:")
print(departments.head())

ORDERS columns:
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  


PRODUCTS columns:
   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart 

In [3]:
print("ORDER PRODUCTS (PRIOR) columns:")
print(order_products_prior.head(10))
print("\n" + "="*80 + "\n")

print("ORDER PRODUCTS (TRAIN) columns:")
print(order_products_train.head(10))

ORDER PRODUCTS (PRIOR) columns:
   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0
5         2       17794                  6          1
6         2       40141                  7          1
7         2        1819                  8          1
8         2       43668                  9          0
9         3       33754                  1          1


ORDER PRODUCTS (TRAIN) columns:
   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1
5         1     

In [4]:
# Check the eval_set distribution
print("Eval set distribution:")
print(orders['eval_set'].value_counts())
print("\n")

# Check how many unique users
print(f"Total unique users: {orders['user_id'].nunique()}")
print(f"Total orders: {len(orders)}")

Eval set distribution:
eval_set
prior    3214874
train     131209
test       75000
Name: count, dtype: int64


Total unique users: 206209
Total orders: 3421083


In [5]:
# Let's verify the connection
# Check if train orders have corresponding products in order_products_train
train_orders_in_orders = orders[orders['eval_set'] == 'train']['order_id'].nunique()
train_orders_in_products = order_products_train['order_id'].nunique()

print(f"Train orders in orders.csv: {train_orders_in_orders}")
print(f"Train orders in order_products_train.csv: {train_orders_in_products}")
print("\n")

# Same for prior
prior_orders_in_orders = orders[orders['eval_set'] == 'prior']['order_id'].nunique()
prior_orders_in_products = order_products_prior['order_id'].nunique()

print(f"Prior orders in orders.csv: {prior_orders_in_orders}")
print(f"Prior orders in order_products_prior.csv: {prior_orders_in_products}")

Train orders in orders.csv: 131209
Train orders in order_products_train.csv: 131209


Prior orders in orders.csv: 3214874
Prior orders in order_products_prior.csv: 3214874


In [6]:
# How many orders per user?
orders_per_user = orders.groupby('user_id')['order_number'].max()

print("Orders per user statistics:")
print(orders_per_user.describe())
print("\n")

# Check a single user's order history as example
sample_user = orders[orders['user_id'] == 1].sort_values('order_number')
print(f"Sample user (user_id=1) order history:")
print(sample_user[['order_id', 'eval_set', 'order_number', 'order_dow', 'days_since_prior_order']])

Orders per user statistics:
count    206209.000000
mean         16.590367
std          16.654774
min           4.000000
25%           6.000000
50%          10.000000
75%          20.000000
max         100.000000
Name: order_number, dtype: float64


Sample user (user_id=1) order history:
    order_id eval_set  order_number  order_dow  days_since_prior_order
0    2539329    prior             1          2                     NaN
1    2398795    prior             2          3                    15.0
2     473747    prior             3          3                    21.0
3    2254736    prior             4          4                    29.0
4     431534    prior             5          4                    28.0
5    3367565    prior             6          2                    19.0
6     550135    prior             7          1                    20.0
7    3108588    prior             8          1                    14.0
8    2295261    prior             9          1                     0.0
9 

In [7]:
# Check eval_set distribution per user
eval_distribution = orders.groupby('user_id')['eval_set'].value_counts().unstack(fill_value=0)
print("Eval set counts per user:")
print(eval_distribution.describe())
print("\n")

# Does each user have exactly 1 train OR 1 test order?
users_with_train = orders[orders['eval_set'] == 'train']['user_id'].nunique()
users_with_test = orders[orders['eval_set'] == 'test']['user_id'].nunique()
users_with_both = len(set(orders[orders['eval_set'] == 'train']['user_id']) & 
                       set(orders[orders['eval_set'] == 'test']['user_id']))

print(f"Users with train order: {users_with_train}")
print(f"Users with test order: {users_with_test}")
print(f"Users with BOTH train and test: {users_with_both}")
print(f"Total users: {orders['user_id'].nunique()}")

Eval set counts per user:
eval_set          prior           test          train
count     206209.000000  206209.000000  206209.000000
mean          15.590367       0.363709       0.636291
std           16.654774       0.481067       0.481067
min            3.000000       0.000000       0.000000
25%            5.000000       0.000000       0.000000
50%            9.000000       0.000000       1.000000
75%           19.000000       1.000000       1.000000
max           99.000000       1.000000       1.000000


Users with train order: 131209
Users with test order: 75000
Users with BOTH train and test: 0
Total users: 206209


In [8]:
# Analyze basket sizes
prior_basket_sizes = order_products_prior.groupby('order_id').size()
train_basket_sizes = order_products_train.groupby('order_id').size()

print("PRIOR orders - basket size statistics:")
print(prior_basket_sizes.describe())
print("\n")

print("TRAIN orders - basket size statistics:")
print(train_basket_sizes.describe())
print("\n")

# Reorder rates
print(f"Prior orders - reorder rate: {order_products_prior['reordered'].mean():.2%}")
print(f"Train orders - reorder rate: {order_products_train['reordered'].mean():.2%}")
print("\n")

# How many unique products total?
print(f"Unique products in prior: {order_products_prior['product_id'].nunique()}")
print(f"Unique products in train: {order_products_train['product_id'].nunique()}")
print(f"Total products in catalog: {len(products)}")

PRIOR orders - basket size statistics:
count    3.214874e+06
mean     1.008888e+01
std      7.525398e+00
min      1.000000e+00
25%      5.000000e+00
50%      8.000000e+00
75%      1.400000e+01
max      1.450000e+02
dtype: float64


TRAIN orders - basket size statistics:
count    131209.000000
mean         10.552759
std           7.932847
min           1.000000
25%           5.000000
50%           9.000000
75%          14.000000
max          80.000000
dtype: float64


Prior orders - reorder rate: 58.97%
Train orders - reorder rate: 59.86%


Unique products in prior: 49677
Unique products in train: 39123
Total products in catalog: 49688


In [9]:
# Product popularity in prior orders
product_counts = order_products_prior['product_id'].value_counts()

print("Product purchase frequency distribution:")
print(product_counts.describe())
print("\n")

print("Top 10 most purchased products:")
top_products = products[products['product_id'].isin(product_counts.head(10).index)]
top_10_with_counts = top_products.merge(
    product_counts.head(10).reset_index(),
    left_on='product_id',
    right_on='product_id'
)
top_10_with_counts.columns = ['product_id', 'product_name', 'aisle_id', 'department_id', 'count']
print(top_10_with_counts[['product_name', 'count']])
print("\n")

# Products that never appear in prior
products_never_purchased = set(products['product_id']) - set(order_products_prior['product_id'])
print(f"Products never purchased in prior: {len(products_never_purchased)}")

Product purchase frequency distribution:
count     49677.000000
mean        652.907563
std        4792.114416
min           1.000000
25%          17.000000
50%          60.000000
75%         260.000000
max      472565.000000
Name: count, dtype: float64


Top 10 most purchased products:
             product_name   count
0  Bag of Organic Bananas  379450
1            Strawberries  142951
2    Organic Strawberries  264683
3    Organic Baby Spinach  241921
4                  Banana  472565
5                   Limes  140627
6      Organic Whole Milk  137905
7    Organic Hass Avocado  213584
8             Large Lemon  152657
9         Organic Avocado  176815


Products never purchased in prior: 11


In [10]:
# Shopping day patterns
print("Orders by day of week:")
print(orders['order_dow'].value_counts().sort_index())
print("\n")

# Shopping hour patterns
print("Orders by hour of day (top 10 hours):")
print(orders['order_hour_of_day'].value_counts().sort_index().head(10))
print("\n")

# Days between orders
print("Days since prior order statistics:")
print(orders['days_since_prior_order'].describe())
print("\n")

# Most common intervals
print("Most common reorder intervals (days):")
print(orders['days_since_prior_order'].value_counts().head(10).sort_index())

Orders by day of week:
order_dow
0    600905
1    587478
2    467260
3    436972
4    426339
5    453368
6    448761
Name: count, dtype: int64


Orders by hour of day (top 10 hours):
order_hour_of_day
0     22758
1     12398
2      7539
3      5474
4      5527
5      9569
6     30529
7     91868
8    178201
9    257812
Name: count, dtype: int64


Days since prior order statistics:
count    3.214874e+06
mean     1.111484e+01
std      9.206737e+00
min      0.000000e+00
25%      4.000000e+00
50%      7.000000e+00
75%      1.500000e+01
max      3.000000e+01
Name: days_since_prior_order, dtype: float64


Most common reorder intervals (days):
days_since_prior_order
1.0     145247
2.0     193206
3.0     217005
4.0     221696
5.0     214503
6.0     240013
7.0     320608
8.0     181717
9.0     118188
30.0    369323
Name: count, dtype: int64


In [11]:
# Merge products with aisles and departments
products_full = products.merge(aisles, on='aisle_id').merge(departments, on='department_id')

# Get product purchases with category info
prior_with_categories = order_products_prior.merge(products_full, on='product_id')

# Top departments
dept_counts = prior_with_categories['department'].value_counts()
print("Top 10 departments by purchase volume:")
print(dept_counts.head(10))
print("\n")

# Top aisles
aisle_counts = prior_with_categories['aisle'].value_counts()
print("Top 10 aisles by purchase volume:")
print(aisle_counts.head(10))
print("\n")

print(f"Total departments: {len(departments)}")
print(f"Total aisles: {len(aisles)}")

Top 10 departments by purchase volume:
department
produce            9479291
dairy eggs         5414016
snacks             2887550
beverages          2690129
frozen             2236432
pantry             1875577
bakery             1176787
canned goods       1068058
deli               1051249
dry goods pasta     866627
Name: count, dtype: int64


Top 10 aisles by purchase volume:
aisle
fresh fruits                     3642188
fresh vegetables                 3418021
packaged vegetables fruits       1765313
yogurt                           1452343
packaged cheese                   979763
milk                              891015
water seltzer sparkling water     841533
chips pretzels                    722470
soy lactosefree                   638253
bread                             584834
Name: count, dtype: int64


Total departments: 21
Total aisles: 134


In [13]:
# First check what columns we actually have
print("order_products_prior columns:")
print(order_products_prior.columns.tolist())
print("\n")

# Need to merge with orders to get user_id
prior_with_users = order_products_prior.merge(
    orders[['order_id', 'user_id']], 
    on='order_id'
)

print("After merge, columns:")
print(prior_with_users.columns.tolist())
print(f"Shape: {prior_with_users.shape}")

order_products_prior columns:
['order_id', 'product_id', 'add_to_cart_order', 'reordered']


After merge, columns:
['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id']
Shape: (32434489, 5)


In [14]:
# User-product interactions
user_product_stats = prior_with_users.groupby(['user_id', 'product_id']).agg({
    'order_id': 'count',  # How many times user bought this product
    'reordered': 'sum'    # How many times it was a reorder
}).rename(columns={'order_id': 'purchase_count', 'reordered': 'reorder_count'}).reset_index()

print("User-product purchase frequency:")
print(user_product_stats['purchase_count'].describe())
print("\n")

# How many unique products does each user buy in their history?
products_per_user = prior_with_users.groupby('user_id')['product_id'].nunique()
print("Unique products per user:")
print(products_per_user.describe())
print("\n")

# How many unique users buy each product?
users_per_product = prior_with_users.groupby('product_id')['user_id'].nunique()
print("Unique users per product:")
print(users_per_product.describe())

User-product purchase frequency:
count    1.330795e+07
mean     2.437226e+00
std      3.554528e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      9.900000e+01
Name: purchase_count, dtype: float64


Unique products per user:
count    206209.000000
mean         64.536238
std          56.592339
min           1.000000
25%          25.000000
50%          48.000000
75%          86.000000
max         726.000000
Name: product_id, dtype: float64


Unique users per product:
count    49677.000000
mean       267.889627
std       1308.788623
min          1.000000
25%         11.000000
50%         35.000000
75%        137.000000
max      73956.000000
Name: user_id, dtype: float64


In [15]:
# Check train set structure
train_with_users = order_products_train.merge(orders[['order_id', 'user_id']], on='order_id')

print(f"Train users: {train_with_users['user_id'].nunique()}")
print(f"Train orders: {train_with_users['order_id'].nunique()}")
print(f"Total train products purchased: {len(train_with_users)}")
print(f"Products with reordered=1: {train_with_users['reordered'].sum()}")
print(f"Reorder rate in train: {train_with_users['reordered'].mean():.2%}")

Train users: 131209
Train orders: 131209
Total train products purchased: 1384617
Products with reordered=1: 828824
Reorder rate in train: 59.86%


In [16]:
# Get all products each train user bought in their prior orders
train_user_ids = train_with_users['user_id'].unique()
train_users_prior = prior_with_users[prior_with_users['user_id'].isin(train_user_ids)]

# Get all prior products per user
prior_products_per_user = train_users_prior.groupby('user_id')['product_id'].apply(set).to_dict()

# Check if train products were in prior
def was_bought_before(row):
    user_id = row['user_id']
    product_id = row['product_id']
    if user_id in prior_products_per_user:
        return product_id in prior_products_per_user[user_id]
    return False

# Sample check on first 10000 rows (full check would take time)
sample = train_with_users.head(10000).copy()
sample['was_in_prior'] = sample.apply(was_bought_before, axis=1)

print("In train orders (sample of 10k products):")
print(f"Products that were in prior: {sample['was_in_prior'].sum()}")
print(f"Products NOT in prior (new): {(~sample['was_in_prior']).sum()}")
print(f"\nReordered flag=1: {sample['reordered'].sum()}")
print(f"Reordered flag=0: {(sample['reordered']==0).sum()}")

In train orders (sample of 10k products):
Products that were in prior: 5781
Products NOT in prior (new): 4219

Reordered flag=1: 5781
Reordered flag=0: 4219


In [17]:
# Get train users' prior purchase history
train_users_prior = prior_with_users[prior_with_users['user_id'].isin(train_user_ids)]
train_user_product_pairs = train_users_prior.groupby(['user_id', 'product_id']).size().reset_index()

print(f"Unique user-product pairs for train users (from prior): {len(train_user_product_pairs)}")
print("\n")

# Now check how many of these were reordered in train
train_reordered_products = train_with_users[train_with_users['reordered']==1][['user_id', 'product_id']].drop_duplicates()

print(f"Products actually reordered in train orders: {len(train_reordered_products)}")

Unique user-product pairs for train users (from prior): 8474661


Products actually reordered in train orders: 828824


In [18]:
# Double check
positive = len(train_reordered_products)
negative = len(train_user_product_pairs) - positive

print(f"Positive examples (reordered): {positive:,} ({positive/len(train_user_product_pairs)*100:.2f}%)")
print(f"Negative examples (not reordered): {negative:,} ({negative/len(train_user_product_pairs)*100:.2f}%)")
print(f"Total: {len(train_user_product_pairs):,}")
print(f"\nClass imbalance ratio: {negative/positive:.1f}:1")

Positive examples (reordered): 828,824 (9.78%)
Negative examples (not reordered): 7,645,837 (90.22%)
Total: 8,474,661

Class imbalance ratio: 9.2:1


In [19]:
# For train orders, what metadata do we have?
train_orders_metadata = orders[orders['eval_set'] == 'train']

print("Train orders metadata (what we know at prediction time):")
print(train_orders_metadata.head())
print("\n")

print("Columns available:")
print(train_orders_metadata.columns.tolist())
print("\n")

# Check if we have timing info
print(f"Orders with day of week: {train_orders_metadata['order_dow'].notna().sum()}")
print(f"Orders with hour: {train_orders_metadata['order_hour_of_day'].notna().sum()}")
print(f"Orders with days_since_prior: {train_orders_metadata['days_since_prior_order'].notna().sum()}")

Train orders metadata (what we know at prediction time):
    order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
10   1187899        1    train            11          4                  8   
25   1492625        2    train            15          1                 11   
49   2196797        5    train             5          0                 11   
74    525192        7    train            21          2                 11   
78    880375        8    train             4          1                 14   

    days_since_prior_order  
10                    14.0  
25                    30.0  
49                     6.0  
74                     6.0  
78                    10.0  


Columns available:
['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']


Orders with day of week: 131209
Orders with hour: 131209
Orders with days_since_prior: 131209


In [20]:
# Check test users - do they have prior history?
test_user_ids = orders[orders['eval_set'] == 'test']['user_id'].unique()
test_users_prior = prior_with_users[prior_with_users['user_id'].isin(test_user_ids)]

print(f"Test users: {len(test_user_ids)}")
print(f"Test users with prior history: {test_users_prior['user_id'].nunique()}")
print(f"Test user-product pairs: {test_users_prior.groupby(['user_id', 'product_id']).ngroups}")

Test users: 75000
Test users with prior history: 75000
Test user-product pairs: 4833292
